In [1]:
import argparse

from utils.logger import get_logger
from utils.arg_parser import Argments
from loader.data_loader import GeneralDataLoaderCls, NbsDataLoaderCls,\
                               GeneralDataLoaderSeg, NbsDataLoaderSeg
from runners.cnn_runner import CnnRunner
from runners.nbs_runner import NbsRunner
from runners.mcd_runner import McdRunner

import torch
print(torch.cuda.is_available())

import sys
print(sys.executable)


True
c:\Users\user\.conda\envs\torch\python.exe


In [2]:
# making instance
argparser = argparse.ArgumentParser()
# adding the values
argparser.add_argument("yaml")
argparser.add_argument("--phase", default="train", type=str) # train / infer 
argparser.add_argument("--index", default=-1, type=int) # meaning ? -> used in file name(spcific version : file1,file2,...) -1 means nothing is added in file name.
argparser.add_argument("--gpus", default="0", type=str)
argparser.add_argument("--local_rank", default=0, type=int) # meaning ? logger level?
# save 
cmd_args = argparser.parse_args(["example"])

arg = Argments(f"scripts/{cmd_args.yaml}.yaml", cmd_args) # stll many funcs to read
setup = arg['setup']
model_path = arg['path/model_path']
logger = get_logger(f"{model_path}/log.txt")



setting the python env
cmd_args.local_rank:  0


TypeError: __init__() missing 1 required positional argument: 'num_classes'

In [3]:
if setup['rank'] == 0:
    logger.info(arg)

model_type = setup['model_type']
dataset = arg['path/dataset']
is_seg = False

2023-05-12 13:26:43,019 - outs/nbs/resnet34/svhn_test/log.txt - INFO - scripts/nbs.yaml
{'path': {'dataset': 'svhn', 'postfix': 'test', 'model_path': 'outs/nbs/resnet34/svhn_test'}, 'setup': {'model_type': 'nbs', 'batch_size': 128, 'cpus': 32, 'gpus': '0', 'seed': 0, 'num_epoch': 2, 'phase': 'train', 'epoch_th': 0, 'num_mc': 5, 'n_a': 512, 'adv_training': False, 'index': -1, 'local_rank': 0, 'rank': 0, 'dist_size': 0}, 'module': {'model': {'name': 'resnet34', 'num_classes': 10, 'dropout_rate': 0.0}, 'optim': {'name': 'SGD', 'lr': 0.05, 'momentum': 0.9, 'nesterov': True, 'weight_decay': 0.001}, 'lr_scheduler': {'name': 'CosineAnnealingLR', 'T_max': 200}, 'loss': ['NbsLoss'], 'loss_args': {'NbsLoss': {'reduction': 'mean'}}, 'loss_weight': [1.0], 'val_metric': 'Accuracy', 'test_metric': 'Accuracy', 'metric_args': {'nlabels': 10, 'reduction': 'none'}}}


In [5]:
if 'nbs' in model_type: # added in n_a (comparing the else loop)
    _data_loader = NbsDataLoaderCls # n_a exists
    data_loader = _data_loader(dataset, setup['batch_size'],
                                setup['n_a'], setup['cpus'], setup['seed'])
    runner = NbsRunner(data_loader, **arg.module, num_epoch=setup['num_epoch'], # core part 
                        logger=logger, model_path=model_path, rank=setup['rank'],
                        epoch_th=setup['epoch_th'], num_mc=setup['num_mc'],
                        adv_training=setup['adv_training'])

Using downloaded and verified file: .svhn\train_32x32.mat
Using downloaded and verified file: .svhn\test_32x32.mat


2023-05-12 12:43:58,712 - outs/nbs/resnet34/svhn_test/log.txt - DEBUG - outs\nbs\resnet34\svhn_test
2023-05-12 12:43:58,713 - outs/nbs/resnet34/svhn_test/log.txt - DEBUG - Failed to load, not existing file


In [ ]:
def train(self):
    self.log("Start to train", 'debug')
    for epoch in range(self.epoch, self.num_epoch):
        self.model.train()
        loader = self.loader.load("train")
        if self.rank == 0:
            t_iter = tqdm(loader, total=self.loader.len,
                            desc=f"[Train {epoch}]")
        else:
            t_iter = loader
        losses = 0
        times = []
        for i, batch in enumerate(t_iter):
            t = time()
            loss = self._train_a_batch(batch) # update the params.
            times += [time() - t]
            losses += loss
            if self.rank == 0:
                t_iter.set_postfix(loss=f"{loss:.4} / {losses/(i+1):.4}")

        self.log(f"[Train] epoch:{epoch} loss:{losses/(i+1)}", 'info')
        print("Batch Training Time : ", np.mean(times))
        self.lr_scheduler.step()
        self.val(epoch)

In [29]:
if setup['phase'] == 'train':
    runner.train() 
    runner.test(is_seg) 
elif setup['phase'] == 'test':
    runner.test(is_seg)


if __name__ == "__main__":
    main()

2023-05-11 17:59:31,483 - outs/nbs/resnet34/svhn_test/log.txt - DEBUG - Start to train
2023-05-11 17:59:31,483 - outs/nbs/resnet34/svhn_test/log.txt - DEBUG - Start to train
[Train 0]:   0%|          | 0/516 [00:23<?, ?it/s]


KeyboardInterrupt: 